In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from rpy2.robjects.packages import importr

POT = importr('POT')

import rpy2.robjects as robjects

In [ ]:
def normal_MC_IS_GPD(n, size, a, sigma):
    data = np.random.normal(0, sigma, (n, size))
    data_IS = data + a
    data = np.mean(data, axis=0)
    
    MC = np.mean(data > a)
    
    IS = np.mean(np.where(np.mean(data_IS, axis=0) > a, 1, 0) * np.exp(-1/2*np.sum(data_IS ** 2, axis=0))/np.exp(-1/2*np.sum((data_IS-a)**2, axis=0)))
    
    u = float(np.quantile(data, 0.99))
    scale, shape = POT.fitgpd(robjects.FloatVector(data), u)[0]
    GPD99 = np.mean(data > u) * POT.pgpd(a, u, scale, shape, False)[0]
    
    u = float(np.quantile(data, 0.999))
    scale, shape = POT.fitgpd(robjects.FloatVector(data), u)[0]
    GPD999 = np.mean(data > u) * POT.pgpd(a, u, scale, shape, False)[0]
    
    u = float(np.quantile(data, 0.9999))
    scale, shape = POT.fitgpd(robjects.FloatVector(data), u)[0]
    GPD9999 = np.mean(data > u) * POT.pgpd(a, u, scale, shape, False)[0]

    return MC, IS, GPD99, GPD999, GPD9999

In [ ]:
num = 1000
n = 10
size = 10 ** 6
a = 2
sigma = 1

MC = np.zeros((num))
IS = np.zeros((num))
GPD99 = np.zeros((num))
GPD999 = np.zeros((num))
GPD9999 = np.zeros((num))

for i in range(num):
    np.random.seed(i)
    MC[i], IS[i], GPD99[i], GPD999[i], GPD9999[i] = normal_MC_IS_GPD(n, size, a, sigma)
np.save(f"sample_mean_results/sample_mean_MC_normal_{num}_{n}_{size}_{a}_{sigma}.npy", MC)
np.save(f"sample_mean_results/sample_mean_IS_normal_{num}_{n}_{size}_{a}_{sigma}.npy", IS)
np.save(f"sample_mean_results/sample_mean_GPD_normal_{num}_{n}_{size}_{a}_{sigma}_99.npy", GPD99)
np.save(f"sample_mean_results/sample_mean_GPD_normal_{num}_{n}_{size}_{a}_{sigma}_999.npy", GPD999)
np.save(f"sample_mean_results/sample_mean_GPD_normal_{num}_{n}_{size}_{a}_{sigma}_9999.npy", GPD9999)

In [ ]:
num = 1000
n = 10
size = 10 ** 6
a = 1.5
sigma = 1

MC = np.load(f"sample_mean_results/sample_mean_MC_normal_{num}_{n}_{size}_{a}_{sigma}.npy")
IS = np.load(f"sample_mean_results/sample_mean_IS_normal_{num}_{n}_{size}_{a}_{sigma}.npy")
GPD99 = np.load(f"sample_mean_results/sample_mean_GPD_normal_{num}_{n}_{size}_{a}_{sigma}_99.npy")
GPD999 = np.load(f"sample_mean_results/sample_mean_GPD_normal_{num}_{n}_{size}_{a}_{sigma}_999.npy")
GPD9999= np.load(f"sample_mean_results/sample_mean_GPD_normal_{num}_{n}_{size}_{a}_{sigma}_9999.npy")

names = ["MC", "IS", "GPD-99", "GPD-999", "GPD-9999"]
results = [MC, IS, GPD99, GPD999, GPD9999]

for name, res in zip(names, results):
    print(f"{name}: mean {np.mean(res):.3e}, sd {np.std(res):.3e}")

In [ ]:
import scipy.stats

def pareto_MC_CMC_GPD(n, size, a, t, alpha):
    data = scipy.stats.pareto.rvs(alpha, scale=t, size=(n, size))
    
    CMC = n * np.mean(1 - scipy.stats.pareto.cdf(np.maximum(np.max(data[:, :size-1], axis=0), n * a - np.sum(data[:, :size-1], axis=0)), alpha, scale=t))
    
    data = np.mean(data, axis=0)
    
    MC = np.mean(data > a)
    
    u = float(np.quantile(data, 0.99))
    scale, shape = POT.fitgpd(robjects.FloatVector(data), u)[0]
    GPD99 = np.mean(data > u) * POT.pgpd(a, u, scale, shape, False)[0]
    
    u = float(np.quantile(data, 0.999))
    scale, shape = POT.fitgpd(robjects.FloatVector(data), u)[0]
    GPD999 = np.mean(data > u) * POT.pgpd(a, u, scale, shape, False)[0]
    
    u = float(np.quantile(data, 0.9999))
    scale, shape = POT.fitgpd(robjects.FloatVector(data), u)[0]
    GPD9999 = np.mean(data > u) * POT.pgpd(a, u, scale, shape, False)[0]

    return MC, CMC, GPD99, GPD999, GPD9999

In [ ]:
num = 1000
n = 10
size = 10 ** 6
a = 5.234375
t = 1
alpha = 5

MC = np.zeros((num))
CMC = np.zeros((num))
GPD99 = np.zeros((num))
GPD999 = np.zeros((num))
GPD9999 = np.zeros((num))

for i in range(num):
    np.random.seed(i)
    MC[i], CMC[i], GPD99[i], GPD999[i], GPD9999[i] = pareto_MC_CMC_GPD(n, size, a, t, alpha)
    
np.save(f"sample_mean_results/sample_mean_MC_pareto_{num}_{n}_{size}_{a}_{t}_{alpha}.npy", MC)
np.save(f"sample_mean_results/sample_mean_CMC_pareto_{num}_{n}_{size}_{a}_{t}_{alpha}.npy", CMC)
np.save(f"sample_mean_results/sample_mean_GPD_pareto_{num}_{n}_{size}_{a}_{t}_{alpha}_99.npy", GPD99)
np.save(f"sample_mean_results/sample_mean_GPD_pareto_{num}_{n}_{size}_{a}_{t}_{alpha}_999.npy", GPD999)
np.save(f"sample_mean_results/sample_mean_GPD_pareto_{num}_{n}_{size}_{a}_{t}_{alpha}_9999.npy", GPD9999)

In [ ]:
num = 1000
n = 10
size = 10 ** 6
a = 5.234375
t = 1
alpha = 5

MC = np.load(f"sample_mean_results/sample_mean_MC_pareto_{num}_{n}_{size}_{a}_{t}_{alpha}.npy")
CMC = np.load(f"sample_mean_results/sample_mean_CMC_pareto_{num}_{n}_{size}_{a}_{t}_{alpha}.npy")
GPD99 = np.load(f"sample_mean_results/sample_mean_GPD_pareto_{num}_{n}_{size}_{a}_{t}_{alpha}_99.npy")
GPD999 = np.load(f"sample_mean_results/sample_mean_GPD_pareto_{num}_{n}_{size}_{a}_{t}_{alpha}_999.npy")
GPD9999= np.load(f"sample_mean_results/sample_mean_GPD_pareto_{num}_{n}_{size}_{a}_{t}_{alpha}_9999.npy")

names = ["MC", "CMC", "GPD-99", "GPD-999", "GPD-9999"]
results = [MC, CMC, GPD99, GPD999, GPD9999]

for name, res in zip(names, results):
    print(f"{name}: mean {np.mean(res):.3e}, sd {np.std(res):.3e}")